In [1]:
#Initial environement of computations


import matplotlib.pyplot as plt
import numpy as np
from kanon.calendars import Calendar, Date
from kanon.calendars.calendars import float_to_hm, hm_to_float
from kanon.tables import HTable
from kanon.units import Sexagesimal, Historical
from kanon.units.precision import set_precision, TruncatureMode, PrecisionMode
import astropy.units as u


from kanon.tables.htable_reader import (
    DISHAS_REQUEST_URL,
    read_historical,
    read_sexag_array,
    read_intsexag_array,
)
from kanon.tables import HTable
import requests

from kanon.units import Sexagesimal, Historical, IntegerAndSexagesimal, Temporal
from kanon.calendars import*

Cal=julian_ad = Calendar.registry["Julian A.D."]

number_reader = {
    "sexagesimal": read_sexag_array,
    "floating sexagesimal": read_sexag_array,
    "integer and sexagesimal": read_intsexag_array,
    "historical": read_historical,
}

In [2]:
#Initial table set

# Historical target table
Hist_table=HTable.read(581, with_units=False)
Hist_table


"Years, opposition",Entries
IntegerAndSexagesimal,Historical
482495 ;,"04s 04 ; 10,50"
491261 ;,"07s 21 ; 46,43"
500027 ;,"11s 09 ; 22,36"
508793 ;,"02s 26 ; 58,29"
517559 ;,"05s 18 ; 45,21"
526325 ;,"09s 06 ; 21,14"
535091 ;,"23 ; 57,07"
543857 ;,"04s 11 ; 33,00"
552623 ;,"07s 29 ; 08,53"


In [3]:
# Formating and initialising the Test tables

Test_recomp=Hist_table.copy()
Delta_recomp=Hist_table.copy()
for i in range(13):
    Test_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)
    Delta_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)

In [4]:
#mean motion parameter from the Parisian Alfonsine Tables
daily_mean_elong=Historical(Sexagesimal("12;11,26,41,37,51,50,39"),7)
radice_elong=Historical(Sexagesimal("3,24;25,49,46,11"),4)
daily_mean_moon_anom=Historical(Sexagesimal("13;3,53,57,30,21,4,13"),7)
radice_moon_anom=Historical(Sexagesimal("3,19;0,14,31,16"),4)
delta_Paris_Toledo=Temporal("0;0,48")
REVO=Historical(Sexagesimal("6,0;0,0"),7)
epoch= Date(Cal, (1, 1, 1))


for i in range(13):
    with set_precision(tmode=TruncatureMode.TRUNC, pmode=7):

        #conversion of date to DN from calendar epoch
        days=int(Hist_table[i][0])-365
        
        #computation of the elongation at date
        elong=(days*daily_mean_elong+radice_elong)%REVO

        #computation of time to conjonction NOT including delta between Paris and Toledo and conversion in DN since epoch
        time=(REVO-elong)/daily_mean_elong
        time=Temporal(time,7)+REVO/(2*daily_mean_elong)
        if time>30:
            time=time-REVO/daily_mean_elong
        else:
            time=time
        time=time.decimal
        time=float(time)+days
        
        #computation of the mean sun (and moon) for that time
        mean_moon_anom=(time*daily_mean_moon_anom+radice_moon_anom)%REVO
        
        
        #filling the Test and Delta tables with the final precision
        with set_precision(tmode=TruncatureMode.ROUND, pmode=2):
            Test_recomp[i][1]=1*mean_moon_anom
            Delta_recomp[i][1]=60*60*(Hist_table[i][1]-Test_recomp[i][1])
Test_recomp

"Years, opposition",Entries
IntegerAndSexagesimal,Historical
482495 ;,"04s 04 ; 10,50"
491261 ;,"07s 21 ; 46,43"
500027 ;,"11s 09 ; 22,36"
508793 ;,"02s 26 ; 58,29"
517559 ;,"05s 18 ; 45,21"
526325 ;,"09s 06 ; 21,14"
535091 ;,"23 ; 57,07"
543857 ;,"04s 11 ; 33,01"
552623 ;,"07s 29 ; 08,54"


In [5]:
# residuals visualisation and statistical summaries 1
B=Delta_recomp.to_pandas()
(B.mean().mean(), B.std().std())

(49845.92307692308, nan)

In [6]:
B.astype(int).style.background_gradient(axis=None)

,Entries
"Years, opposition",
482495 ;,0
491261 ;,0
500027 ;,0
508793 ;,0
517559 ;,0
526325 ;,0
535091 ;,0
543857 ;,-1
552623 ;,-1


In [7]:
#csv export of the recomputed table
B=Test_recomp.to_pandas()
compression_opts = dict(method='zip',archive_name='Mean syzygy mean moon years opposition.csv')
B.to_csv('Mean syzygy moon years opposition.zip', index=False, compression=compression_opts)